# Booknlp

## Set key options manually

### Which novel?

In [ ]:
NOVEL_TXT=None
while not NOVEL_TXT:
    NOVEL_TXT=input('Paste either a URL to a text file (e.g. from Project Gutenberg) or paste an entire novel\'s text here.').strip()

In [ ]:
NOVEL_TITLE=None
while not NOVEL_TITLE:
    NOVEL_TITLE=input('Enter a title for the novel you wish to analyze').strip()

In [ ]:
import requests
NOVEL_TXT=requests.get(NOVEL_TXT.strip()).text.strip() if NOVEL_TXT.startswith('http') else NOVEL_TXT.strip()
print(f'Loaded novel with {len(NOVEL_TXT.split())} words')

In [1]:
NOVEL_TITLE="BleakHouse"
NOVEL_URL="http://www.gutenberg.org/cache/epub/1023/pg1023.txt"
# NOVEL_TITLE="SensSens"
# NOVEL_URL="http://www.gutenberg.org/cache/epub/161/pg161.txt"
# NOVEL_URL='TomJones'
# NOVEL_URL='http://www.gutenberg.org/files/6593/6593-0.txt'
# NOVEL_TITLE='Pamela'
# NOVEL_URL='http://www.gutenberg.org/files/6124/6124-0.txt'
# NOVEL_TITLE='Middlemarch'
# NOVEL_URL='http://www.gutenberg.org/files/145/145-0.txt'
# NOVEL_TITLE='PortraitOfLady'
# NOVEL_URL=['http://www.gutenberg.org/files/2833/2833-0.txt','http://www.gutenberg.org/files/2834/2834-0.txt']

### Parameters

In [2]:
MIN_COUNT_TOTAL=5

In [3]:
# !pwd

## Other setup

In [4]:
# set paths
import os
# PATH_ROOT=os.path.abspath(os.expanduser('~/'))
PATH_ROOT='/home/jovyan/work/digital-humanities-cookbook/character-networks'
PATH_TOOLS=os.path.join(PATH_ROOT,'tools')
PATH_NOVELS=os.path.join(PATH_ROOT,'texts')
if not os.path.exists(PATH_TOOLS): os.makedirs(PATH_TOOLS)
PATH_TO_BOOKNLP=os.path.join(PATH_TOOLS,'book-nlp')
URL_CORENLP='http://nlp.stanford.edu/software/stanford-corenlp-4.1.0.zip'
MODEL_FN='stanford-corenlp-4.1.0-models.jar'
PATH_TO_BOOKNLP_BINARY=os.path.abspath(os.path.join(PATH_TO_BOOKNLP,'runjava'))

In [5]:
os.chdir(PATH_TOOLS)
if not os.path.exists(PATH_TO_BOOKNLP):
    !git clone https://github.com/dbamman/book-nlp
PATH_BOOKNLP_MODELS=os.path.join(PATH_TO_BOOKNLP,'lib',MODEL_FN)
if not os.path.exists(PATH_BOOKNLP_MODELS):
    !wget $URL_CORENLP
    corenlp_fn=URL_CORENLP.split('/')[-1]
    corenlp_dir=f'{corenlp_fn.split(".zip")[0]}'
    !unzip $corenlp_fn
    ifnfn=f'{corenlp_fn.split(".zip")[0]}/{MODEL_FN}'
    !mv $ifnfn $PATH_BOOKNLP_MODELS
    !rm $corenlp_fn
    !rm -rf $corenlp_dir

In [6]:
PATH_JAVA_HOME=os.path.join(PATH_TOOLS,'jdk-15.0.1')
PATH_JAVA_BIN=os.path.join(PATH_JAVA_HOME,'bin','java')
if not os.path.exists(PATH_JAVA_BIN):
    !wget https://download.java.net/java/GA/jdk15.0.1/51f4f36ad4ef43e39d0dfdbaf6549e32/9/GPL/openjdk-15.0.1_linux-x64_bin.tar.gz
    !tar xvf openjdk-15.0.1_linux-x64_bin.tar.gz
    !rm openjdk-15.0.1_linux-x64_bin.tar.gz

In [8]:
# imports
import os,sys
import pandas as pd
import networkx as nx
import dynetx as dn
from collections import Counter

In [10]:
## Download novel
import os
PATH_NOVEL=os.path.join(PATH_NOVELS,NOVEL_TITLE)
if not os.path.exists(PATH_NOVEL): os.makedirs(PATH_NOVEL)
ofn_novel=os.path.join(PATH_NOVEL,f'{NOVEL_TITLE}.txt')
ofn=os.path.join(PATH_NOVEL,f'data.parses.{NOVEL_TITLE}.jsonl')
ofn_meta=os.path.join(PATH_NOVEL,f'data.charmeta.{NOVEL_TITLE}.csv')
ofn_meta_anno=os.path.join(PATH_NOVEL,f'data.charmeta.{NOVEL_TITLE}.anno.csv')
os.chdir(PATH_NOVEL)
print(os.getcwd())

if not os.path.exists(ofn_novel):
    if type(NOVEL_URL)!=list:
        !wget "$NOVEL_URL" -O "$ofn_novel"
    else:
        for i,url in enumerate(NOVEL_URL):
            fn=url.split('/')[-1]
            !wget "$url"
            !cat $fn >> "$ofn_novel" 
            !rm $fn

/home/jovyan/work/digital-humanities-cookbook/character-networks/texts/BleakHouse


## Parse

In [13]:
def parse_text(path_txt):
    if not path_txt: return    
    path_out=os.path.splitext(path_txt)[0]+'.booknlp'
    path_toks=os.path.join(path_out,'tokens.txt')
    cmd=f'export JAVA_HOME="{PATH_JAVA_HOME}" && export PATH="{PATH_JAVA_HOME}/bin:$PATH" && cd {PATH_TO_BOOKNLP} && ./runjava novels/BookNLP -doc {path_txt} -printHTML -p {path_out} -tok {path_toks} -f'
    print('>>',cmd)
    !{cmd} #os.system(cmd)
    os.rename(os.path.join(path_out,'book.id.html'), os.path.join(path_out,'parsed.html'))
    os.rename(os.path.join(path_out,'book.id.book'), os.path.join(path_out,'parsed.json'))

In [12]:
# Parse! Will take a few minutes...
parse_text(ofn_novel)

>> export JAVA_HOME="/home/jovyan/work/digital-humanities-cookbook/character-networks/tools/jdk-15.0.1" && export PATH="/home/jovyan/work/digital-humanities-cookbook/character-networks/tools/jdk-15.0.1/bin:$PATH" && cd /home/jovyan/work/digital-humanities-cookbook/character-networks/tools/book-nlp && ./runjava novels/BookNLP -doc /home/jovyan/work/digital-humanities-cookbook/character-networks/texts/BleakHouse/BleakHouse.txt -printHTML -p /home/jovyan/work/digital-humanities-cookbook/character-networks/texts/BleakHouse/BleakHouse.booknlp -tok /home/jovyan/work/digital-humanities-cookbook/character-networks/texts/BleakHouse/BleakHouse.booknlp/tokens.txt -f
Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words-distsim.tagger ... done [1.7 sec].
Adding annotator lemma
Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [2.0 sec]

## Analysis

In [ ]:
paths_json = [os.path.join(root,'parsed.json') for root,dirs,fns in sorted(os.walk(egdir)) if 'parsed.json' in fns]
len(paths_json)

72

In [ ]:
def read_parsed_json(path_parsed_json):
    import json,os
    from collections import defaultdict,Counter
    #json.loads(path_parsed_json)
    dat=json.load(open(path_parsed_json))
    keys=[]
    nullchar=defaultdict(Counter)
    text_id=path_parsed_json.split('/')[-3]
    for char in dat['characters']:
        if not char['names']: continue
        names=[x['n'] for x in char['names']]
        
        chardx={'name':names[0], 'id':char['id'], 'names':names, 'text_id':text_id}
        num=0
        for key in ['agent','patient','poss','mod','speaking']:
            chardx['num_'+key]=len(char[key])
            num+=chardx['num_'+key]
            chardx['words_'+key]=words=[]
            for event in char[key]:
                if 'w' in event:
                    wtxt=event['w']
                    wlist=word_tokenize(wtxt) if ' ' in wtxt else [wtxt]
                    wlist=[w.lower() for w in wlist if w and w[0].isalpha()]
                    words+=wlist
        chardx['num']=num
        yield chardx

In [ ]:
ld = [dx for path in paths_json for dx in read_parsed_json(path)]

In [ ]:
df=pd.DataFrame(ld)
# df

In [ ]:
import gender_guesser.detector as gender
gd = gender.Detector()

def guess_gender(name,gd=gd):
    if name.startswith('Mr.') or name.startswith('Mr '): return 'male'
    if name.startswith('Ms.') or name.startswith('Ms '): return 'female'
    if name.startswith('Mrs.') or name.startswith('Mrs '): return 'female'    
    if name.startswith('Miss') or name.startswith('Miss '): return 'female'
    gend=gd.get_gender(name)
    return gend

In [ ]:
df['gender']=df.name.apply(guess_gender)

In [ ]:
bad_names={'Chesney Wold','Lincoln'}
df=df[~df.name.isin(bad_names)]

In [ ]:
# df.name.value_counts()
# df.gender.value_counts()

## Manual intervention?

In [ ]:
dfm=df.groupby(['name','gender']).sum().reset_index()
dfm

,name,gender,id,num_agent,num_patient,num_poss,num_mod,num_speaking,num
0,Ada,female,323,241,72,151,29,0,493
1,Alfred,male,18,2,0,1,0,0,3
2,Allan,male,14,140,37,71,9,0,257
3,Arethusa,unknown,14,1,0,0,0,0,1
4,Bart,male,18,9,1,10,4,0,24
...,...,...,...,...,...,...,...,...,...
179,Volumina,unknown,7,0,0,0,1,0,1
180,W. Grubble,unknown,2,1,0,0,0,0,1
181,Watt,unknown,17,9,0,1,2,0,12
182,William Buffy,unknown,2,3,0,0,0,0,3


In [ ]:
dfm['name_real']=dfm['name']
dfm['other']=''
dfm['notes']=''
dfm['race']=''
dfm['class']=''
dfm_save=dfm[['name','name_real','num','gender','race','class','other','notes']].sort_values('num',ascending=False)
dfm_save

,name,name_real,num,gender,race,class,other,notes
161,Richard,Richard,1026,male,,,,
97,Mr. Bucket,Mr. Bucket,731,male,,,,
123,Mr. Tulkinghorn,Mr. Tulkinghorn,673,male,,,,
104,Mr. Guppy,Mr. Guppy,670,male,,,,
106,Mr. Jarndyce,Mr. Jarndyce,536,male,,,,
...,...,...,...,...,...,...,...,...
77,Melchisedech,Melchisedech,0,unknown,,,,
78,Michael Jackson,Michael Jackson,0,unknown,,,,
25,Cursitor Street,Cursitor Street,0,unknown,,,,
85,Miss Jogg,Miss Jogg,0,female,,,,


In [ ]:
dfm_save.to_csv(ofn_meta,index=False)
if not os.path.exists(ofn_meta_anno):
    dfm_save.to_csv(ofn_meta_anno,index=False)

In [ ]:
### EDIT CSV and save to data.charmeta.anno.csv

In [ ]:
# !jupyter labextension install spreadsheet-editor

In [ ]:
ifn=ofn_meta_anno if os.path.exists(ofn_meta_anno) else ofn_meta
dfm=pd.read_csv(ifn).fillna('').rename(columns={'num':'num_total'})
okcols=['name']+list(set(df.columns) - set(dfm.columns))
df=df[okcols].merge(dfm,on='name')
df=df[(df.name_real.apply(lambda x: x[0].isalpha())) & (df.num_total>=MIN_COUNT_TOTAL)]
df['name']=df['name_real']

## Dynamic network

In [ ]:
def make_dyn_charnet(df,min_mentions_in_chap=3):
    dg = dn.DynGraph(edge_removal=False)
    for chapter,chap_df in sorted(df.groupby('text_id')):
        named=dict(zip(chap_df.name,chap_df.num))
        names=[n for n in chap_df.name if named[n]>=min_mentions_in_chap]
#         for i,n2 in enumerate(names[1:]):
#             n1=names[i]
#             dg.add_interaction(u=n1,v=n2,t=int(chapter))
        for n1 in sorted(names):
            for n2 in sorted(names):
                if n2<=n1: continue
                dg.add_interaction(u=n1,v=n2,t=int(chapter))
    return dg

In [ ]:
def to_static(dg):
    import networkx as nx
    g=nx.Graph()
    for a,b,d in dg.edges(data=True):
        d['weight']=len(d['t'])
        g.add_edge(a,b,**d)
    return g

In [ ]:
def layout(g):
    from fa2 import ForceAtlas2
    forceatlas2 = ForceAtlas2(
        # Behavior alternatives
        outboundAttractionDistribution=True,  # Dissuade hubs
        linLogMode=False,  # NOT IMPLEMENTED
        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
        edgeWeightInfluence=1.0,

        # Performance
        jitterTolerance=1.0,  # Tolerance
        barnesHutOptimize=True,
        barnesHutTheta=1.2,
        multiThreaded=False,  # NOT IMPLEMENTED

        # Tuning
        scalingRatio=2.0,
        strongGravityMode=False,
        gravity=1.0,

        # Log
        verbose=True
    )
    
    pos = forceatlas2.forceatlas2_networkx_layout(g, pos=None, iterations=2000)
    return pos

In [ ]:
def drawnet_nx(g,ofn='net.png',pos=None,sized={},size_factor=10000):
    from matplotlib import pyplot as plt
    fig = plt.figure(figsize=(20,20))#,facecolor=(0, 0, 0))
    nodelist=g.nodes()
    labels=dict((n,n) for n in nodelist)
    node_size=[sized[n]*size_factor if n in sized and sized[n] else 300 for n in nodelist]
    
    edgelist=list(g.edges())
    edge_size=[g[a][b]['weight'] for a,b in edgelist]
    edge_size=[g[a][b]['weight'] for a,b in edgelist]
    
    nx.draw_networkx(
        g,
        pos=pos,
        labels=labels,
        nodelist=nodelist,
        node_size=node_size,
        edgelist=edgelist,
        width=edge_size,
        font_color='black',
        font_weight='bold',
        font_size=14,
        node_color='gray',
        edge_color='teal'
    )
    plt.savefig(ofn)
    plt.close()

In [ ]:
def drawnets(dg):
    import math
    from tqdm import tqdm
    g=to_static(dg)
    #pos=nx.spring_layout(g,k=5/math.sqrt(g.order()))
    pos=layout(g)
    for t in tqdm(dg.temporal_snapshots_ids()):
        cg=to_static(dg.time_slice(t_from=t,t_to=t))
        g_sofar=to_static(dg.time_slice(t_from=0,t_to=t))
        sized=nx.betweenness_centrality(g_sofar)
        #pos=nx.spring_layout(g_sofar)
        drawnet_nx(g_sofar,ofn=f'figs/net-{str(t).zfill(2)}.png',pos=pos,sized=sized)

In [ ]:
dg=make_dyn_charnet(df,min_mentions_in_chap=5)
drawnets(dg)

  2%|▏         | 1/65 [00:00<00:08,  7.96it/s]

BarnesHut Approximation  took  0.06  seconds
Repulsion forces  took  0.12  seconds
Gravitational forces  took  0.01  seconds
Attraction forces  took  0.01  seconds
AdjustSpeedAndApplyForces step  took  0.02  seconds


100%|██████████| 65/65 [00:19<00:00,  3.29it/s]


In [ ]:
# drawnet_nx(to_static(dg.time_slice(t_from=2,t_to=2)), pos=pos)